In [ ]:
import cv2
import numpy as np

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

def draw_lines(img, lines, color=(0, 0, 255), thickness=3):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)



In [ ]:
# Open a video capture object
cap = cv2.VideoCapture('your_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

    # Define the region of interest vertices
    height, width = edges.shape
    roi_vertices = [(0, height), (width / 2, height / 2), (width, height)]
    masked_edges = region_of_interest(edges, np.array([roi_vertices], np.int32))

    # Perform Hough line transformation to detect lines in the image
    lines = cv2.HoughLinesP(
        masked_edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=100, maxLineGap=50
    )

    # Create a blank image to draw the detected lanes on
    line_img = np.zeros_like(frame)

    draw_lines(line_img, lines)

    # Overlay the detected lanes on the original frame
    combined_img = cv2.addWeighted(frame, 0.8, line_img, 1, 0)

    cv2.imshow("Lane Detection", combined_img)

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close all windows
cap.release()
cv2.destroyAllWindows()
